In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, ElementClickInterceptedException
from tqdm import tqdm

options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=C:/Users/cherp/AppData/Local/Google/Chrome/User Data/Default')

driver = webdriver.Chrome(options=options)

with open('saved_links_stroitelstvo_i_remont.txt', 'r') as file:
    urls = file.readlines()

urls = [url.strip() for url in urls]

data = []

output_messages = []
urls = urls[:]
# Перебор каждого URL-а из списка
for url in tqdm(urls):
    url = url.strip()
    retry = True

    while retry:
        try:
            driver.get(url)

            time.sleep(2)
            second_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "(//button[@type='button'])[2]"))
            )
            
            WebDriverWait(driver, 10).until(
                EC.invisibility_of_element_located((By.CLASS_NAME, "scroll-to-top-button"))
            )
            
            driver.execute_script("arguments[0].click();", second_button)

            if len(driver.window_handles) > 1:
                driver.switch_to.window(driver.window_handles[1])
                driver.close()
                driver.switch_to.window(driver.window_handles[0])

            wait = WebDriverWait(driver, 10)
            wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.flocktory-widget-overlay")))

            filter_control = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.catalog-listing-filter-control"))
            )

            buttons = filter_control.find_elements(By.XPATH, ".//button[@type='button']")

            time.sleep(2)
            
            if len(buttons) >= 2:
                buttons[1].click()

                time.sleep(2)

                found_seller = False

                for attempt in range(5):
                    seller_divs = driver.find_elements(By.CSS_SELECTOR, "div.checkbox-list-with-selected__heading")
                    for div in seller_divs:
                        if "Продавец" in div.text:
                            seller_div = div
                            found_seller = True
                            break
                    if found_seller:
                        break
                    else:
                        scroll_value = 100
                        inner_scroll = driver.find_element(By.CLASS_NAME, "filters")
                        driver.execute_script("arguments[0].scrollTop += arguments[1];", inner_scroll, scroll_value)

                if found_seller:
                    button = seller_div.find_element(By.CSS_SELECTOR, "button")
                    driver.execute_script("arguments[0].click();", button)

                    filter_values_listing = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "filter-values-list__listing"))
                    )

                    time.sleep(2)

                    checkbox_texts = WebDriverWait(filter_values_listing, 10).until(
                        EC.presence_of_all_elements_located((By.CLASS_NAME, "checkbox-text"))
                    )

                    # Вывести текст каждого элемента и добавить в список data
                    for text in checkbox_texts:
                        data.append({'Ссылка': url, 'Название Продавца': text.text})
                else:
                    output_messages.append(f"Продавец не найден на странице {url}")
                retry = False
        except (TimeoutException, StaleElementReferenceException, ElementClickInterceptedException):
            output_messages.append(f"Не удалось найти элемент на странице {url}, переход к предыдущей ссылке")
            retry = False  # Переход к предыдущей ссылке

df = pd.DataFrame(data)

df.to_excel('продавцы_saved_links_elektronika_buf.xlsx', index=False)

with open('output_saved_links_elektronika.txt', 'w') as file:
    for message in output_messages:
        file.write(message + '\n')

driver.quit()


In [ ]:
import os

# Путь к существующему файлу Excel
existing_excel_file = 'продавцы_saved_links_elektronika2709.xlsx'

# Если файл уже существует, загрузить его в DataFrame
if os.path.isfile(existing_excel_file):
    existing_df = pd.read_excel(existing_excel_file)
else:
    # Если файла нет, создать пустой DataFrame
    existing_df = pd.DataFrame()

new_df = pd.DataFrame(data)

# Объединяем существующий DataFrame и новый DataFrame
combined_df = pd.concat([existing_df, new_df], ignore_index=True)

combined_df.drop_duplicates(inplace=True)

combined_df.to_excel(existing_excel_file, index=False)

with open('output_saved_links_elektronika.txt', 'w') as file:
    for message in output_messages:
        file.write(message + '\n')

driver.quit()


In [ ]:
import pandas as pd

file1 = 'продавцы_saved_links_elektronika1809_1.xlsx'
file2 = 'продавцы_saved_links_elektronika1809_2.xlsx'
file3 = 'продавцы_saved_links_elektronika1909.xlsx'
file4 = 'продавцы_saved_links_elektronika2409.xlsx'

df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)
df3 = pd.read_excel(file3)
df4 = pd.read_excel(file4)

result = pd.concat([df1, df2, df3, df4], axis=0, ignore_index=True)

def flag_column(row, df, column_name):
    if row[column_name] in df[column_name].values:
        return '+'
    else:
        return '-'

result['01.11.23'] = result.apply(lambda row: flag_column(row, df1, 'Название Продавца'), axis=1)
result['02.11.23'] = result.apply(lambda row: flag_column(row, df2, 'Название Продавца'), axis=1)
result['03.11.23'] = result.apply(lambda row: flag_column(row, df3, 'Название Продавца'), axis=1)
result['04.11.23'] = result.apply(lambda row: flag_column(row, df4, 'Название Продавца'), axis=1)

result.to_excel('merged_data_with_flags.xlsx', index=False)
